<a href="https://colab.research.google.com/github/GuanRuLai/Python-Deep-Learning/blob/main/Vanilla_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
# import dataset

import os

if not os.path.isdir("HappyML"):
    os.system("git clone https://github.com/cnchi/HappyML.git")

df = "Meals.csv"

if not os.path.isfile(df):
  os.system("wget https://raw.githubusercontent.com/cnchi/datasets/master/" + df)

In [23]:
# set customizable constants

train_size = 0.75

win_size = 2
sample_step = 1
win_moving = 1

data_batch = 10

In [24]:
import HappyML.preprocessor as pp
import numpy as np

dataset = pp.dataset(file="Meals.csv")

# keep "meal" column only
dataset = pp.onehot_encoder(dataset, columns=[1]).iloc[:, 1:]
print(dataset)

    Meal_大魯麵  Meal_煎餃  Meal_自助餐  Meal_黃金炒飯
0          0        1         0          0
1          0        0         1          0
2          1        0         0          0
3          0        0         1          0
4          0        0         0          1
5          1        0         0          0
6          0        1         0          0
7          0        0         1          0
8          1        0         0          0
9          0        0         1          0
10         0        0         0          1
11         1        0         0          0
12         0        1         0          0
13         0        0         1          0
14         1        0         0          0
15         0        0         1          0
16         0        0         0          1
17         1        0         0          0
18         0        1         0          0
19         0        0         1          0
20         1        0         0          0
21         0        0         1          0
22         

In [25]:
# keep the One Hot Encoding Mapping
mapper = [s[5:] for s in list(dataset.columns)]

# convert dataset from DataFrame as NDArray
dataset = dataset.values
train_data, test_data = np.split(dataset, [int(train_size * len(dataset))])
# print(train_data)
# print(test_data)

In [28]:
# generate training and testing set

from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator

train_set = TimeseriesGenerator(
        data=train_data, # dependent variable X
        targets=train_data,  # independent variable Y
        length=win_size,
        sampling_rate=sample_step,
        stride=win_moving,
        batch_size=data_batch)

test_set = TimeseriesGenerator(
        data=test_data, # dependent variable X
        targets=test_data, # independent variable Y
        length=win_size,
        sampling_rate=sample_step,
        stride=win_moving,
        batch_size=data_batch)

In [31]:
# create Model

from tensorflow.keras.models import Sequential
from tensorflow.keras import layers

model = Sequential()
model.add(layers.SimpleRNN(units=10, input_shape=(win_size, dataset.shape[1])))
model.add(layers.Dense(units=dataset.shape[1], activation="softmax"))

In [32]:
# model compiling

model.compile(optimizer="rmsprop", loss="categorical_crossentropy", metrics=["acc"])

In [33]:
# model training
# run multiple times of RNN can raise up the accuracy

model.fit(train_set, epochs=25)

Epoch 1/25
3/3 [==============================] - 1s 6ms/step - loss: 1.3146 - acc: 0.2857
Epoch 2/25
3/3 [==============================] - 0s 4ms/step - loss: 1.2728 - acc: 0.2857
Epoch 3/25
3/3 [==============================] - 0s 5ms/step - loss: 1.2493 - acc: 0.4286
Epoch 4/25
3/3 [==============================] - 0s 5ms/step - loss: 1.2273 - acc: 0.5238
Epoch 5/25
3/3 [==============================] - 0s 4ms/step - loss: 1.2107 - acc: 0.6190
Epoch 6/25
3/3 [==============================] - 0s 4ms/step - loss: 1.1975 - acc: 0.6667
Epoch 7/25
3/3 [==============================] - 0s 4ms/step - loss: 1.1801 - acc: 0.8095
Epoch 8/25
3/3 [==============================] - 0s 4ms/step - loss: 1.1660 - acc: 0.8095
Epoch 9/25
3/3 [==============================] - 0s 4ms/step - loss: 1.1539 - acc: 0.8095
Epoch 10/25
3/3 [==============================] - 0s 4ms/step - loss: 1.1391 - acc: 0.8095
Epoch 11/25
3/3 [==============================] - 0s 6ms/step - loss: 1.1273 - acc: 0.80

In [34]:
# model evaluation

test_loss, test_acc = model.evaluate(test_set)
print("Loss of Test Set:", test_loss)
print("Accuracy of Test Set:", test_acc)

1/1 [==============================] - 0s 188ms/step - loss: 0.9432 - acc: 0.8333
Loss of Test Set: 0.9432282447814941
Accuracy of Test Set: 0.8333333134651184


In [35]:
# answer prediction

Y_pred = np.argmax(model.predict(test_set), axis=-1)
Y_real = np.argmax(test_set[0][1], axis=-1) # Y of testing set
print("Prediction:", [mapper[i] for i in Y_pred])
print("Real Value:", [mapper[i] for i in Y_real])

1/1 [==============================] - 0s 167ms/step
Prediction: ['自助餐', '大魯麵', '大魯麵', '黃金炒飯', '大魯麵', '煎餃']
Real Value: ['自助餐', '大魯麵', '自助餐', '黃金炒飯', '大魯麵', '煎餃']


In [37]:
# user input prediction

from tensorflow.keras.utils import to_categorical

meals = []
meals += [mapper.index(input("請問你前天吃什麼："))]
meals += [mapper.index(input("請問你昨天吃什麼："))]

meals = np.reshape(to_categorical(meals, dataset.shape[1]), (1, win_size, dataset.shape[1]))
meal_today = np.argmax(model.predict(meals), axis=-1)
print("我猜你今天會吃：", mapper[meal_today[0]])

請問你前天吃什麼：煎餃
請問你昨天吃什麼：自助餐
1/1 [==============================] - 0s 16ms/step
我猜你今天會吃： 大魯麵
